In [1]:
import nltk
from nltk.corpus import wordnet as wn
import numpy as np
import pandas as pd

from nltk import word_tokenize          
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel, euclidean_distances, manhattan_distances

import json

In [2]:
stop_words = set(stopwords.words('english')) 

# Interface lemma tokenizer from nltk with sklearn
class LemmaTokenizer:
    ignore_tokens = [',', '.', ';', ':', '"', '``', "''", '`', '-']
    def __init__(self):
        self.wnl = WordNetLemmatizer()
    def __call__(self, doc):
        return [self.wnl.lemmatize(t) for t in word_tokenize(doc) if t not in self.ignore_tokens]

# Lemmatize the stop words
tokenizer=LemmaTokenizer()
token_stop = tokenizer(' '.join(stop_words))

In [3]:
def extract_ids(dicdata):
    ids = []
    for i in dict_data:
        mea_id = i['id']
        ids.append(mea_id)
    return ids

def extract_meanings(dicdata):
    meaning_list = []
    for i in dict_data:
        meaning = i['meaning'] 
        meaning_list.append(meaning)
    return meaning_list

def extract_pos_meanings(dicdata):
    pos_meaning = []
    for i in dict_data:
        pos = i['fl']
        meaning = i['meaning'] 
        pos_mea = pos + ' '+ meaning # POS + meaning
        pos_meaning.append(pos_mea)
    return pos_meaning

def extract_isteacher(dicdata):
    isteacher = []
    for i in dict_data:
        is_tea = i['isTeacher']
        isteacher.append(is_tea)
    return isteacher

### Cosine Similarity

In [4]:
# Load Data and store
jsonfile= open('data_600.json','r',encoding='utf8') # removed word aimless
data= jsonfile.read() 

# Parse and Extract word, paragraph, pos and meanings
obj = json.loads(data)

data = obj['data'] #[:10]
print("Total words: ",len(data)) # Total # of words
count=0
# Store accuracy and MRR in blank lists
acc_score=[] 
mrr_score = []

# Parse data and find best meaning
for each_word_data in data:
    word = each_word_data['word']
    paragraph = each_word_data['paragraph']
    dict_data = each_word_data['dictionaryData']
    ids = extract_ids(dict_data)
    meanings = extract_meanings(dict_data)
    pos_meanings = extract_pos_meanings(dict_data)
    #print("\npos_meanings: ",pos_meanings)
    isteacher = extract_isteacher(dict_data)
    
    
    # Create TF-idf model
    vectorizer = TfidfVectorizer(stop_words=token_stop, 
                                  tokenizer=tokenizer)
    doc_vectors = vectorizer.fit_transform([paragraph] + meanings)

    # Calculate similarity
    cosine_similarities = linear_kernel(doc_vectors[0:1], doc_vectors).flatten()
    document_scores = [item.item() for item in cosine_similarities[1:]] # Find document scores


    # Output the document similarity results
    zipped = list(zip(ids, meanings, document_scores, isteacher))
    features = sorted(zipped, key = lambda x: x[2], reverse = True)

    print("\n\nLookup word: ",word)
    ranked_meanings = pd.DataFrame(features, columns = ['id', 'meaning','score','isTeacher'])
    ranked_meanings['isTeacher'] = ranked_meanings['isTeacher'].fillna(0)
    print(ranked_meanings)
    
    count +=1
    print(count)

    
    
    # Find accuracy
    if (ranked_meanings.iloc[0]['isTeacher'] ==1) & (ranked_meanings.iloc[0]['score'] !=0):
        acc_score.append(1)
    else:
        acc_score.append(0)
       
        
    # Find MRR score
    istea_index = ranked_meanings[ranked_meanings['isTeacher']==1].index[0]+1
    pred_rank =1/istea_index
    mrr_score.append(pred_rank)

    
# Return Accuracy and MRR    
#print(acc_score)
avg = sum(acc_score)/len(acc_score)
print("The accuracy score is ", round(avg,4))    

mrr = np.sum(mrr_score)/ len(mrr_score)
print("The MRR is ", round(mrr,4)) 


Total words:  600


Lookup word:  circumvent 
     id                                            meaning  score  isTeacher
0  3230  to avoid being stopped by (something, such as ...    0.0          1
1


Lookup word:  addict
     id                                            meaning     score  \
0  5885                    to cause addiction in (someone)  0.039549   
1  3401  a person who is not able to stop taking drugs ...  0.025158   
2  5884  one exhibiting a compulsive, chronic, physiolo...  0.000000   

   isTeacher  
0        0.0  
1        0.0  
2        1.0  
2


Lookup word:  frustrated
     id                                            meaning  score  isTeacher
0  5886  feeling, showing, or characterized by frustrat...    0.0        1.0
1  5887                   to balk or defeat in an endeavor    0.0        0.0
3


Lookup word:  manslaughter
     id                                            meaning  score  isTeacher
0  2554  the crime of killing a person without intendin...



Lookup word:  truants
     id                                            meaning     score  \
0  2739     a student who misses school without permission  0.068070   
1  5960  to idle away time especially while playing truant  0.043329   
2  5961  one who shirks duty; especially : one who stay...  0.042711   

   isTeacher  
0        1.0  
1        0.0  
2        0.0  
32


Lookup word:  swagger
     id                                            meaning     score  \
0  2744  to walk in a very confident way : to walk with...  0.042273   
1  2743  a way of walking or behaving that shows you ha...  0.000000   
2  5964             marked by elegance or showiness : posh  0.000000   
3  5965  a short light stick usually covered with leath...  0.000000   
4  5962  to conduct oneself in an arrogant or supercili...  0.000000   
5  5963  an arrogantly self-confident way of walking : ...  0.000000   

   isTeacher  
0        0.0  
1        1.0  
2        0.0  
3        0.0  
4        0.0  
5    

     id                                            meaning  score  isTeacher
0  2794  to free (someone) from someone else's control ...    0.0        1.0
1  2795                                 yourself or itself    0.0        0.0
2  5548                 the act or process of emancipating    0.0        0.0
3  5549         the act or process of emancipating oneself    0.0        0.0
67


Lookup word:  massacres
     id                                            meaning     score  \
0  2693                 the violent killing of many people  0.021265   
1  2694  to violently kill (a group of people) : slaughter  0.015445   

   isTeacher  
0        1.0  
1        0.0  
68


Lookup word:  sharecroppers
     id                                            meaning     score  \
0  3399  a farmer especially in the southern U.S. who r...  0.011245   
1  6022  a tenant farmer especially in the southern U.S...  0.009112   

   isTeacher  
0        1.0  
1        0.0  
69


Lookup word:  eerily
   



Lookup word:  jarring
     id                                            meaning  score  isTeacher
0  2850  to have a harsh or unpleasant effect on someon...    0.0        1.0
1  6054  having a harshly concussive, disagreeable, or ...    0.0        0.0
2  6055  to have a harshly disagreeable or disconcertin...    0.0        0.0
102


Lookup word:  immense
     id                                            meaning  score  isTeacher
0  6056  marked by greatness especially in size or degr...    0.0          1
103


Lookup word:  escorting
     id                                            meaning  score  isTeacher
0  3422  to go with (someone or something) to give prot...    0.0        1.0
1  6057                          to accompany as an escort    0.0        0.0
104


Lookup word:  prophet
     id                                            meaning     score  \
0  6059       someone who says that bad things will happen  0.028456   
1  6058  one who utters divinely inspired revelations



Lookup word:  gesturing
     id                                            meaning  score  isTeacher
0  6708  direct or invite (someone) to move somewhere s...    0.0          1
1  6128                                  to make a gesture    0.0          1
141


Lookup word:  Occupied
     id                                 meaning     score  isTeacher
0  6129  to engage the attention or energies of  0.071222        1.0
1  6130                   lived in by the owner  0.000000        0.0
142


Lookup word:  resented
     id                                      meaning  score  isTeacher
0  6131  to feel or express annoyance or ill will at    0.0          1
143


Lookup word:  revising 
     id                                            meaning  score  isTeacher
0  6132  to look over again in order to correct or improve    0.0          1
144


Lookup word:  aggressive
     id                                            meaning  score  isTeacher
0  3833  ready and willing to fight, argue, 

...



Lookup word:  grimaced
     id                                            meaning  score  isTeacher
0  6605  to distort one's face in an expression usually...    0.0          1
434


Lookup word:  pluck 
     id                                            meaning  score  isTeacher
0  6691                         to pull or pick off or out    0.0        1.0
1  6693  to pull part of (something) with one's fingers...    0.0        0.0
2  6694           to become brave enough (to do something)    0.0        0.0
3  6692          an act or instance of plucking or pulling    0.0        0.0
435


Lookup word:  absorbed
     id                                            meaning     score  \
0  6704  absorbed in one's own thoughts, activities, or...  0.126109   
1  2945  to take in (something, such as a liquid) in a ...  0.073504   
2  6702  having one's attention wholly engaged or occupied  0.000000   
3  2946       only caring about and interested in yourself  0.000000   
4  6703  to take in 



Lookup word:  dispatched
     id                                            meaning  score  isTeacher
0  4722  to send away promptly or rapidly to a particul...    0.0        1.0
1  6061  to send off or away with promptness or speed; ...    0.0        0.0
473


Lookup word:  martyr
     id                                            meaning  score  isTeacher
0  3047  a person who is killed or who suffers greatly ...    0.0        1.0
1  6168  to put to death for adhering to a belief, fait...    0.0        0.0
2  6169  to act like someone who deserves admiration or...    0.0        0.0
3  3048  to kill (someone) for refusing to give up a be...    0.0        0.0
4  6167  a person who voluntarily suffers death as the ...    0.0        0.0
474


Lookup word:  transcendent
     id                 meaning  score  isTeacher
0  4809  exceeding usual limits    0.0          1
475


Lookup word:  absurd
     id                                            meaning     score  \
0  6772  a state or c



Lookup word:  audible
     id                                            meaning     score  \
0  6832  too quiet or obscured to be heard clearly : in...  0.099813   
1  6831                                 to call an audible  0.081130   
2  6829                    heard or capable of being heard  0.000000   
3  6830  a substitute offensive or defensive play calle...  0.000000   

   isTeacher  
0        1.0  
1        0.0  
2        0.0  
3        0.0  
501


Lookup word:  commenced
     id                                            meaning  score  isTeacher
0  4749  to bring or come into activity, being, or oper...    0.0          1
502


Lookup word:  malicious
     id                                            meaning  score  isTeacher
0  2849  having or showing a desire to cause harm to an...    0.0          1
503


Lookup word:  dehumanizing
     id                                            meaning     score  \
0  6836  to deprive (someone or something) of human qua...  0.02970



Lookup word:  intervention
     id                                          meaning     score  isTeacher
0  1904     to come or occur between two times or events  0.053202        1.0
1  6884  the act or an instance of intervening : such as  0.033776        0.0
530


Lookup word:  intervene
     id                                            meaning     score  \
0  6885  to occur, fall, or come between points of time...  0.042353   

   isTeacher  
0          1  
531


Lookup word:  Overt
     id                  meaning  score  isTeacher
0  4761  open to view : manifest    0.0          1
532


Lookup word:  assault
     id                                            meaning     score  \
0  4611                            to make an assault upon  0.023929   
1  4612  any of various automatic or semiautomatic rifl...  0.011509   
2  4610    a violent physical or verbal attack : onslaught  0.000000   

   isTeacher  
0        0.0  
1        0.0  
2        1.0  
533


Lookup word:  bartere



Lookup word:  teeters
     id             meaning  score  isTeacher
0  4881  to move unsteadily    0.0          1
575


Lookup word:  clatters
     id                                            meaning  score  isTeacher
0  4882          to make or cause to make a rattling sound    0.0        1.0
1  4883  a rattling sound (as of hard bodies striking t...    0.0        0.0
576


Lookup word:  pushiness
     id                                            meaning  score  isTeacher
0  6970  aggressive often to an objectionable degree : ...    0.0          1
577


Lookup word:  eviction
     id                                            meaning  score  isTeacher
0  6971  to recover (property) from a person by legal p...    0.0          1
578


Lookup word:  disparity
     id                                            meaning  score  isTeacher
0  2049  a noticeable and often unfair difference betwe...    0.0        1.0
1  6333  a noticeable and usually significant differenc...    0.0        

### Euclidean distance

In [5]:
count=0
# Store accuracy and MRR in blank lists
acc_score=[] 
mrr_score = []

# Parse data and find best meaning
for each_word_data in data:
    word = each_word_data['word']
    paragraph = each_word_data['paragraph']
    dict_data = each_word_data['dictionaryData']
    ids = extract_ids(dict_data)
    meanings = extract_meanings(dict_data)
    pos_meanings = extract_pos_meanings(dict_data)
    #print("\npos_meanings: ",pos_meanings)
    isteacher = extract_isteacher(dict_data)
    
    
    # Create TF-idf model
    vectorizer = TfidfVectorizer(stop_words=token_stop, 
                                  tokenizer=tokenizer)
    doc_vectors = vectorizer.fit_transform([paragraph] + meanings)

    # Calculate similarity
    euclidean_distance = euclidean_distances(doc_vectors[0:1], doc_vectors).flatten()
    document_scores = [item.item() for item in euclidean_distance[1:]] # find document scores

    # Output the document similarity results
    zipped = list(zip(ids, meanings, document_scores, isteacher))
    features = sorted(zipped, key = lambda x: x[2], reverse = False)

    print("\n\nLookup word: ",word)
    ranked_meanings = pd.DataFrame(features, columns = ['id', 'meaning','score','isTeacher'])
    ranked_meanings['isTeacher'] = ranked_meanings['isTeacher'].fillna(0)
    print(ranked_meanings)
    
    count +=1
    print(count)

    
    
    # Find accuracy
    #if ranked_meanings.iloc[0]['isTeacher'] ==1:
    if (ranked_meanings.iloc[0]['isTeacher'] ==1) & (ranked_meanings.iloc[0]['score'] !=0):
        acc_score.append(1)
    else:
        acc_score.append(0)
        
    # Find MRR score
    istea_index = ranked_meanings[ranked_meanings['isTeacher']==1].index[0]+1
    pred_rank =1/istea_index
    mrr_score.append(pred_rank)

    
# Return Accuracy and MRR    
#print(acc_score)
avg = sum(acc_score)/len(acc_score)
print("The accuracy score is ", round(avg,4))    

mrr = np.sum(mrr_score)/ len(mrr_score)
print("The MRR is ", round(mrr,4)) 




Lookup word:  circumvent 
     id                                            meaning     score  \
0  3230  to avoid being stopped by (something, such as ...  1.414214   

   isTeacher  
0          1  
1


Lookup word:  addict
     id                                            meaning     score  \
0  5885                    to cause addiction in (someone)  1.385966   
1  3401  a person who is not able to stop taking drugs ...  1.396311   
2  5884  one exhibiting a compulsive, chronic, physiolo...  1.414214   

   isTeacher  
0        0.0  
1        0.0  
2        1.0  
2


Lookup word:  frustrated
     id                                            meaning     score  \
0  5886  feeling, showing, or characterized by frustrat...  1.414214   
1  5887                   to balk or defeat in an endeavor  1.414214   

   isTeacher  
0        1.0  
1        0.0  
3


Lookup word:  manslaughter
     id                                            meaning     score  \
0  2554  the crime of killing



Lookup word:  constitutes
     id                 meaning     score  isTeacher
0  5944  make up, form, compose  1.414214          1
25


Lookup word:  befuddled
     id                                         meaning     score  isTeacher
0  2502         unable to think clearly : very confused  1.414214        0.0
1  5945  utterly confused or puzzled : deeply perplexed  1.414214        1.0
2  5946   to muddle or stupefy with or as if with drink  1.414214        0.0
26


Lookup word:  damning
     id                                            meaning     score  \
0  3477                                 yourself or itself  1.000000   
1  5949  causing or leading to one's own condemnation o...  1.414214   
2  3476  —used to show that you are angry or annoyed at...  1.414214   
3  5948  to condemn to a punishment or fate; especially...  1.414214   
4  5947                                 bringing damnation  1.414214   

   isTeacher  
0        0.0  
1        1.0  
2        0.0  
3        



Lookup word:  infirmity
     id                                            meaning     score  \
0  2728  the quality or state of being weak or ill espe...  1.396542   
1  5671               the quality or state of being infirm  1.414214   

   isTeacher  
0        1.0  
1        0.0  
51


Lookup word:  enchanting
    id                                            meaning     score  isTeacher
0  607  to attract and hold the attention of (someone)...  1.405385          1
52


Lookup word:  evocative
     id                                            meaning     score  \
0  2727  bringing thoughts, memories, or feelings into ...  1.414214   

   isTeacher  
0          1  
53


Lookup word:  shuttered
     id                                    meaning     score  isTeacher
0  2730          to cover (a window) with shutters  1.414214        1.0
1  2729  having shutters or having closed shutters  1.414214        0.0
54


Lookup word:  abstinence
     id                                      



Lookup word:  chagrined
     id                                            meaning     score  \
0  2820            frustrated or annoyed : feeling chagrin  1.414214   
1  6044  to vex or unsettle by disappointing or humilia...  1.414214   

   isTeacher  
0        1.0  
1        0.0  
91


Lookup word:  eavesdrop
     id                                        meaning     score  isTeacher
0  6045  to listen secretly to what is said in private  1.414214          1
92


Lookup word:  Delta
     id                                            meaning     score  \
0  2796     the fourth letter of the Greek alphabet—Δ or ƍ  1.414214   
1  6046  low-lying region in NW Miss., extending eastwa...  1.414214   

   isTeacher  
0        0.0  
1        1.0  
93


Lookup word:  idealism
     id                                            meaning     score  \
0  6047  the practice of forming ideals or living under...  1.414214   

   isTeacher  
0          1  
94


Lookup word:  distracted
     id    



Lookup word:  compromised
     id                                            meaning     score  \
0  5541  made vulnerable (as to attack or misuse) by un...  1.377884   
1  5542          to come to agreement by mutual concession  1.414214   

   isTeacher  
0          0  
1          1  
125


Lookup word:  endorse
     id                                            meaning     score  \
0  6091  to write on the back of; especially : to sign ...  1.393961   
1  3637  to publicly or officially say that you support...  1.414214   

   isTeacher  
0        0.0  
1        1.0  
126


Lookup word:  dignity
     id                                            meaning     score  \
0  2888  a way of appearing or behaving that suggests s...  1.414214   
1  6093  formal reserve or seriousness of manner, appea...  1.414214   
2  6094  suitable for someone who is considered less im...  1.414214   

   isTeacher  
0        1.0  
1        0.0  
2        0.0  
127


Lookup word:  veracity
     id       

...



Lookup word:  overtaken
     id                                            meaning     score  \
0  6720                                   to catch up with  1.414214   
1  6721  forced to be changed because of something that...  1.414214   

   isTeacher  
0        0.0  
1        1.0  
454


Lookup word:  detaching
     id                                            meaning     score  \
0  6722  to separate especially from a larger mass and ...  1.414214   

   isTeacher  
0          1  
455


Lookup word:  ethereal
     id                                         meaning     score  isTeacher
0  6723  of or relating to the regions beyond the earth  1.414214          1
456


Lookup word:  Obligingly
     id                         meaning     score  isTeacher
0  4788  willing to do favors : helpful  1.414214          1
457


Lookup word:  pedagogy
     id                                            meaning     score  \
0  4462  the art, science, or profession of teaching; e...  1.414214  



Lookup word:  hesitated
     id                              meaning     score  isTeacher
0  6783  to hold back in doubt or indecision  1.414214          1
485


Lookup word:  trudged
     id                                            meaning     score  \
0  6785  to walk or march steadily and usually laboriously  1.414214   

   isTeacher  
0          1  
486


Lookup word:  flitted
     id                                            meaning     score  \
0  6786  to pass quickly or abruptly from one place or ...  1.414214   

   isTeacher  
0          1  
487


Lookup word:  marijuana
     id                                            meaning     score  \
0  6801     opposed to or restricting the use of marijuana  1.381231   
1  6802  marijuana that is available only by prescripti...  1.390719   
2  6800  the psychoactive dried resinous flower buds an...  1.414214   
3  3702  the dried leaves and flowers of the hemp plant...  1.414214   

   isTeacher  
0        0.0  
1        0.0  




Lookup word:  ethics
     id                                            meaning     score  \
0  6861         a set of rules about good and bad behavior  1.383910   
1  6860  a system of ethics by which acts are judged wi...  1.386304   
2  6862  an ethic that stresses the virtue of hard work...  1.393509   
3  2124  rules of behavior based on ideas about what is...  1.395349   
4  6859  the discipline dealing with what is good and b...  1.414214   
5  6863  a belief in work as a moral good : a set of va...  1.414214   

   isTeacher  
0        0.0  
1        0.0  
2        0.0  
3        1.0  
4        0.0  
5        0.0  
521


Lookup word:  aspiration
     id                                            meaning     score  \
0  6810  a strong desire to achieve something high or g...  1.414214   
1  4734     audible breath that accompanies a speech sound  1.414214   

   isTeacher  
0          1  
1          0  
522


Lookup word:  bygone
     id         meaning     score  isTeacher
0 



Lookup word:  compelled
     id                                      meaning     score  isTeacher
0  6927           to feel required (to do something)  1.414214        1.0
1  6926  to drive or urge forcefully or irresistibly  1.414214        0.0
562


Lookup word:  vanished
     id                                 meaning     score  isTeacher
0  6928  to pass quickly from sight : disappear  1.414214          1
563


Lookup word:  Delta
     id                                            meaning     score  \
0  6046  low-lying region in NW Miss., extending eastwa...  1.414214   
1  2796     the fourth letter of the Greek alphabet—Δ or ƍ  1.414214   

   isTeacher  
0        1.0  
1        0.0  
564


Lookup word:  forsake
     id                                            meaning     score  \
0  6931             to renounce or turn away from entirely  1.392829   
1  6932  —used to refer to making art for no other reas...  1.414214   
2  6934  —used to make a statement or question more f



Lookup word:  alternative
      id                                            meaning     score  \
0   3481  something that can be chosen instead of someth...  1.374065   
1   6161  alternative music that blends elements of conv...  1.386094   
2   6153  a proposition or situation offering a choice b...  1.392777   
3   6158  a U.S. federal income tax that was originally ...  1.404447   
4   6155  usable power (such as heat or electricity) tha...  1.414214   
5   6154                                        alt-country  1.414214   
6   6159  music that is produced by performers who are o...  1.414214   
7   6160  pop music that has broad appeal but that is pr...  1.414214   
8   6156  a fuel for internal combustion engines that is...  1.414214   
9   3480                    offering or expressing a choice  1.414214   
10  6157  any of various systems of healing or treating ...  1.414214   

    isTeacher  
0         1.0  
1         0.0  
2         0.0  
3         0.0  
4         0.0  

### Manhattan Distance

In [6]:
count=0
# Store accuracy and MRR in blank lists
acc_score=[] 
mrr_score = []

# Parse data and find best meaning
for each_word_data in data:
    word = each_word_data['word']
    paragraph = each_word_data['paragraph']
    dict_data = each_word_data['dictionaryData']
    ids = extract_ids(dict_data)
    meanings = extract_meanings(dict_data)
    pos_meanings = extract_pos_meanings(dict_data)
    #print("\npos_meanings: ",pos_meanings)
    isteacher = extract_isteacher(dict_data)
    
    
    # Create TF-idf model
    vectorizer = TfidfVectorizer(stop_words=token_stop, 
                                  tokenizer=tokenizer)
    doc_vectors = vectorizer.fit_transform([paragraph] + meanings)

    # Calculate similarity
    manhattan_distance = manhattan_distances(doc_vectors[0:1], doc_vectors).flatten()
    document_scores = [item.item() for item in manhattan_distance[1:]] # Find document scores

    # Output the document similarity results
    zipped = list(zip(ids, meanings, document_scores, isteacher))
    features = sorted(zipped, key = lambda x: x[2], reverse = False)

    print("\n\nLookup word: ",word)
    ranked_meanings = pd.DataFrame(features, columns = ['id', 'meaning','score','isTeacher'])
    ranked_meanings['isTeacher'] = ranked_meanings['isTeacher'].fillna(0)
    print(ranked_meanings)
    
    count +=1
    print(count)

    
    
    # Find accuracy
    if (ranked_meanings.iloc[0]['isTeacher'] ==1) & (ranked_meanings.iloc[0]['score'] !=0):
        acc_score.append(1)
    else:
        acc_score.append(0)
        
    # Find MRR score
    istea_index = ranked_meanings[ranked_meanings['isTeacher']==1].index[0]+1
    pred_rank =1/istea_index
    mrr_score.append(pred_rank)

    
# Return Accuracy and MRR    
#print(acc_score)
avg = sum(acc_score)/len(acc_score)
print("The accuracy score is ", round(avg,4))    

mrr = np.sum(mrr_score)/ len(mrr_score)
print("The MRR is ", round(mrr,4)) 




Lookup word:  circumvent 
     id                                            meaning      score  \
0  3230  to avoid being stopped by (something, such as ...  11.127417   

   isTeacher  
0          1  
1


Lookup word:  addict
     id                                            meaning      score  \
0  5885                    to cause addiction in (someone)  10.492664   
1  3401  a person who is not able to stop taking drugs ...  10.713905   
2  5884  one exhibiting a compulsive, chronic, physiolo...  11.639736   

   isTeacher  
0        0.0  
1        0.0  
2        1.0  
2


Lookup word:  frustrated
     id                                            meaning      score  \
0  5887                   to balk or defeat in an endeavor   9.953973   
1  5886  feeling, showing, or characterized by frustrat...  10.221922   

   isTeacher  
0        0.0  
1        1.0  
3


Lookup word:  manslaughter
     id                                            meaning     score  \
0  2554  the crime o



Lookup word:  convictions
     id                                            meaning      score  \
0  5937       the courage to do what one believes is right   9.393076   
1  2576  the ability to do something that you know is d...   9.989898   
2  2575  the act of proving that a person is guilty of ...  10.390938   
3  5936  the act or process of finding a person guilty ...  10.737569   

   isTeacher  
0        1.0  
1        0.0  
2        0.0  
3        0.0  
23


Lookup word:  resentments
     id                                            meaning      score  \
0  5938  a feeling of indignant displeasure or persiste...  11.123119   

   isTeacher  
0          1  
24


Lookup word:  constitutes
     id                 meaning     score  isTeacher
0  5944  make up, form, compose  9.332051          1
25


Lookup word:  befuddled
     id                                         meaning     score  isTeacher
0  5946   to muddle or stupefy with or as if with drink  6.314627        0.0
1  

Lookup word:  marquees
     id                                            meaning     score  \
0  6001  a large tent set up for an outdoor party, rece...  9.965002   

   isTeacher  
0          1  
48


Lookup word:  loiterers
     id                                            meaning     score  \
0  2722  to remain in an area when you do not have a pa...  9.319251   

   isTeacher  
0          1  
49


Lookup word:  hovering
     id                                            meaning     score  \
0  2723  to float in the air without moving in any dire...  9.388587   

   isTeacher  
0          1  
50


Lookup word:  infirmity
     id                                            meaning     score  \
0  5671               the quality or state of being infirm  7.758756   
1  2728  the quality or state of being weak or ill espe...  8.508698   

   isTeacher  
0        0.0  
1        1.0  
51


Lookup word:  enchanting
    id                                            meaning     score  isTea

Lookup word:  enlightenment
     id                                            meaning     score  \
0  2698  the state of having knowledge or understanding...  9.097626   

   isTeacher  
0          1  
82


Lookup word:  refuge
     id                                            meaning     score  \
0  2803  shelter or protection from danger or trouble —...  9.501827   

   isTeacher  
0          1  
83


Lookup word:  smug
     id                                            meaning    score  isTeacher
0  2661  having or showing the annoying quality of peop...  7.54108          1
84


Lookup word:  lurched
     id                                      meaning      score  isTeacher
0  3140  to make a sudden sideways or forward motion  11.940392          1
85


Lookup word:  mediocre
     id        meaning      score  isTeacher
0  3141  not very good  10.704324          1
86


Lookup word:  wherewithal
     id                                            meaning      score  \
0  2813  the mo



Lookup word:  deceptively
     id                                            meaning     score  \
0  6076  tending or having power to cause someone to ac...  8.208554   

   isTeacher  
0          1  
117


Lookup word:  eagerness
     id                                            meaning     score  \
0  6077  marked by enthusiastic or impatient desire or ...  9.466306   
1  2865  very excited and interested : feeling a strong...  9.734284   

   isTeacher  
0        0.0  
1        1.0  
118


Lookup word:  evicted
     id                                            meaning     score  \
0  2877                to force (someone) to leave a place  9.030925   
1  6078  to recover (property) from a person by legal p...  9.227753   

   isTeacher  
0        1.0  
1        0.0  
119


Lookup word:  reluctant
     id                                            meaning     score  \
0  3984  feeling or showing doubt about doing something...  7.943267   
1  6079  feeling or showing aversion, hes

...

Lookup word:  abolitionists
     id                                            meaning     score  \
0  4555  a person who is in favor of abolition especial...  6.641726   
1  6773  a person who wants to stop or abolish slavery ...  7.049329   

   isTeacher  
0          0  
1          1  
477


Lookup word:  palpable
     id                 meaning     score  isTeacher
0  3262  obvious and noticeable  3.863703          1
478


Lookup word:  incredulity
     id                                            meaning     score  \
0  4728  the quality or state of being incredulous : di...  5.152963   

   isTeacher  
0          1  
479


Lookup word:  torrent
     id                                            meaning     score  \
0  6777                                         torrential  6.114896   
1  6776                     a tumultuous outpouring : rush  6.846947   
2  4729  a violent or rushing stream of a liquid (as wa...  7.943323   

   isTeacher  
0        0.0  
1        1.0  
2     

Lookup word:  bail
     id                                            meaning     score  \
0  4565                                         great evil  6.407593   
1  4568   to spring or cause to spring into the air : leap  6.674569   
2  4564                             a supporting half hoop  6.725430   
3  4567  to jump with a parachute from an airplane in f...  6.993379   
4  4560       a container used to remove water from a boat  7.222686   
5  4569  the giving of oneself or something into the po...  7.405076   
6  4563  to entrust (personal property) to another for ...  7.918130   
7  4561  to remove (water) from a boat by dipping and t...  8.183765   
8  4566  a large bundle of goods; especially : one clos...  8.310004   
9  4562  security that guarantees the appearance of a p...  8.596172   

   isTeacher  
0        0.0  
1        0.0  
2        0.0  
3        0.0  
4        0.0  
5        0.0  
6        0.0  
7        0.0  
8        0.0  
9        1.0  
506


Lookup word:  imp

0          1  
531


Lookup word:  Overt
     id                  meaning     score  isTeacher
0  4761  open to view : manifest  9.426888          1
532


Lookup word:  assault
     id                                            meaning     score  \
0  4611                            to make an assault upon  7.839227   
1  4610    a violent physical or verbal attack : onslaught  8.491248   
2  4612  any of various automatic or semiautomatic rifl...  9.436461   

   isTeacher  
0        0.0  
1        1.0  
2        0.0  
533


Lookup word:  bartered
     id                                            meaning     score  \
0  6891  to trade by exchanging one commodity for anoth...  8.940619   

   isTeacher  
0          1  
534


Lookup word:  homicides
     id                                            meaning     score  \
0  4756                         a person who kills another  8.422487   
1  6868  the deliberate and unlawful killing of one per...  9.131773   

   isTeacher  
0       

Lookup word:  compelled
     id                                      meaning     score  isTeacher
0  6926  to drive or urge forcefully or irresistibly  5.605551        0.0
1  6927           to feel required (to do something)  5.841619        1.0
562


Lookup word:  vanished
     id                                 meaning     score  isTeacher
0  6928  to pass quickly from sight : disappear  5.605551          1
563


Lookup word:  Delta
     id                                            meaning     score  \
0  2796     the fourth letter of the Greek alphabet—Δ or ƍ  8.383077   
1  6046  low-lying region in NW Miss., extending eastwa...  8.975436   

   isTeacher  
0        0.0  
1        1.0  
564


Lookup word:  forsake
     id                                            meaning      score  \
0  6938                           for no particular reason   9.090574   
1  6931             to renounce or turn away from entirely   9.540203   
2  6933             —used to express surprise or ann

Lookup word:  distracted
     id                                            meaning     score  \
0  6048  having one's thoughts or attention drawn away ...  8.705036   
1  6049  to draw or direct (something, such as someone'...  8.930232   

   isTeacher  
0        1.0  
1        0.0  
596


Lookup word:  alternative
      id                                            meaning      score  \
0   6154                                        alt-country   6.953167   
1   3480                    offering or expressing a choice   7.674172   
2   3481  something that can be chosen instead of someth...   8.488794   
3   6153  a proposition or situation offering a choice b...   8.744326   
4   6155  usable power (such as heat or electricity) tha...   9.086463   
5   6161  alternative music that blends elements of conv...   9.119579   
6   6156  a fuel for internal combustion engines that is...   9.446640   
7   6157  any of various systems of healing or treating ...   9.807658   
8   6158  a U.S